In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import requests as req
from alpha_vantage.timeseries import TimeSeries as ts
import seaborn as sns
import io

print("loading successful")

/Users/sant/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


loading successful


In [2]:
api_key = open('avkey.txt').read()

In [3]:
def get_stock_data(symbol, interval='daily', output_size='full'):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_{interval.upper()}&symbol={symbol}&outputsize={output_size}&apikey={api_key}&datatype=csv'
    response = req.get(url)
    data = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(data))
    return df

In [4]:
symbol = 'AAPL'
stock_data = get_stock_data(symbol, interval='daily', output_size='full')
stock_data.head(7)

,timestamp,open,high,low,close,volume
0,2024-10-14,228.70,231.73,228.60,231.30,38813987
1,2024-10-11,229.30,229.41,227.34,227.55,31759188
2,2024-10-10,227.78,229.50,227.17,229.04,28183544
3,2024-10-09,225.23,229.75,224.83,229.54,33591091
4,2024-10-08,224.30,225.98,223.25,225.77,31855693
5,2024-10-07,224.50,225.69,221.33,221.69,39505354
6,2024-10-04,227.90,228.00,224.13,226.80,37345098


In [5]:
stock_data.describe()

,open,high,low,close,volume
count,6278.000000,6278.000000,6278.000000,6278.000000,6.278000e+03
mean,174.184031,176.111269,172.158361,174.194251,3.312505e+07
std,147.173143,148.385986,145.761517,147.109716,3.058917e+07
min,12.990000,13.190000,12.720000,13.120000,7.025000e+05
25%,78.292500,79.455000,76.905000,78.222500,1.207345e+07
50%,138.970000,140.725050,136.915000,138.985000,2.460056e+07
75%,201.890000,204.270000,199.645000,202.385000,4.432081e+07
max,702.410000,705.070000,699.570000,702.100000,3.326072e+08


In [6]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6278 entries, 0 to 6277
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  6278 non-null   object 
 1   open       6278 non-null   float64
 2   high       6278 non-null   float64
 3   low        6278 non-null   float64
 4   close      6278 non-null   float64
 5   volume     6278 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 294.4+ KB


In [7]:
stock_data['timestamp'] = pd.to_datetime(stock_data['timestamp'])
stock_data.set_index('timestamp', inplace=True)
stock_data.sort_index(inplace=True)

stock_data.head(7)

,open,high,low,close,volume
timestamp,,,,,
1999-11-01,80.00,80.69,77.37,77.62,2487300
1999-11-02,78.00,81.69,77.31,80.25,3564600
1999-11-03,81.62,83.25,81.00,81.50,2932700
1999-11-04,82.06,85.37,80.62,83.62,3384700
1999-11-05,84.62,88.37,84.00,88.31,3721500
1999-11-08,87.75,97.73,86.75,96.37,8490400
1999-11-09,94.37,94.50,88.00,89.62,7224800


In [8]:
metric = stock_data[['close']]

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(metric)

In [10]:
def create_sequences(data, seq_length):
    X = list() 
    y = list()
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

In [11]:
seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

In [12]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [13]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4974, 60, 1) (4974,) (1244, 60, 1) (1244,)


In [14]:
import torch.nn as nn
import torch.optim as optim